<a href="https://colab.research.google.com/github/KTB2110/PortfolioProjects/blob/Projects/SQlitePart1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('video_games.db')
print("Opened database successfully");

Opened database successfully


In [10]:
def ExecuteQuery(query):
  conn = sqlite3.connect('video_games.db')

  df = pd.read_sql_query(query, conn)
  return df

1) List the sales for the top three ranked video games. Include the attributes in the expected output below.

In [17]:
query = """
select v.rank,v.name,v.year,v.genre, s.sales,r.name 
from video_game v join games_sales s 
on v.rank = s.game_rank join region r 
on s.region_id = r.id 
where v.rank > 0 AND v.rank < 4 order by v.rank;"""

ExecuteQuery(query)

,rank,name,year,genre,sales,name
0,1,Wii Sports,2006,Sports,41.49,North America
1,1,Wii Sports,2006,Sports,29.02,European Union
2,1,Wii Sports,2006,Sports,3.77,Japan
3,1,Wii Sports,2006,Sports,8.46,Rest of the World
4,2,Super Mario Bros.,1985,Platform,29.08,North America
5,2,Super Mario Bros.,1985,Platform,3.58,European Union
6,2,Super Mario Bros.,1985,Platform,6.81,Japan
7,2,Super Mario Bros.,1985,Platform,0.77,Rest of the World
8,3,Mario Kart Wii,2008,Racing,15.85,North America
9,3,Mario Kart Wii,2008,Racing,12.88,European Union


2) List the game publisher and the platform manufacturer for each video game ranked in the top twenty. In this question, you need to include the developer table twice (e.g., developer d1, developer d2) and join one copy with video_game and the other with platform.


In [18]:
query = """
select v.rank, v.name, p.platform_name, d.name as game_publisher, d2.name as platform_manufacturer 
from video_game v join platform p on v.platform_id = p.platform_id 
join developer d on v.publisher_id = d.id 
join developer d2 on p.developer_id= d2.id 
where rank < 21;"""

ExecuteQuery(query)

,rank,name,platform_name,game_publisher,platform_manufacturer
0,1,Wii Sports,Wii,Nintendo,Nintendo
1,2,Super Mario Bros.,NES,Nintendo,Nintendo
2,3,Mario Kart Wii,Wii,Nintendo,Nintendo
3,4,Wii Sports Resort,Wii,Nintendo,Nintendo
4,5,Pokemon Red/Pokemon Blue,GB,Nintendo,Nintendo
5,6,Tetris,GB,Nintendo,Nintendo
6,7,New Super Mario Bros.,DS,Nintendo,Nintendo
7,8,Wii Play,Wii,Nintendo,Nintendo
8,9,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo
9,10,Duck Hunt,NES,Nintendo,Nintendo


3) List the total sales for each video game and discard the detailed sales in each region.

In [19]:
query = """
select v.rank, v.name, p.platform_name, d.name as publisher, d2.name as p_man, SUM(s.sales) 
from video_game v join platform p on v.platform_id = p.platform_id 
join developer d on v.publisher_id = d.id 
join developer d2 on p.developer_id= d2.id 
join games_sales s on v.rank = s.game_rank 
where rank < 4 group by v.rank, v.name, p.platform_name, d.name, d2.name;	
"""

ExecuteQuery(query)

,rank,name,platform_name,publisher,p_man,SUM(s.sales)
0,1,Wii Sports,Wii,Nintendo,Nintendo,82.74
1,2,Super Mario Bros.,NES,Nintendo,Nintendo,40.24
2,3,Mario Kart Wii,Wii,Nintendo,Nintendo,35.83


4) List all genres with the number of video games in each one.

In [20]:
query = """
select genre, count(*) 
from video_game 
group by genre;	"""

ExecuteQuery(query)

,genre,count(*)
0,Action,7
1,Fighting,1
2,Misc,3
3,Platform,8
4,Puzzle,2
5,Racing,5
6,Role-Playing,8
7,Shooter,10
8,Simulation,2
9,Sports,4


5) List genre-year pairs with the count of games and total sales in each pair. Keep only genre-year pair with two games or more.

In [21]:
query = """
select v.genre, v.year, count(DISTINCT(v.rank)) as games_in_genre, sum(s.sales) 
from video_game v join games_sales s 
on v.rank = s.game_rank 
group by v.genre, v.year 
having games_in_genre > 1
order by sum(sales);	"""

ExecuteQuery(query)

,genre,year,games_in_genre,sum(s.sales)
0,Shooter,2010,2,27.37
1,Shooter,2012,2,27.78
2,Shooter,2011,2,28.23
3,Simulation,2005,2,37.03
4,Action,2013,2,37.77
5,Sports,2009,2,55.00


6) List pairs of games that have the same genre, work on the same platform, and published by the same publisher in the same year.

In [22]:
query = """
select v1.rank, v1.name, v2.rank, v2.name 
from video_game v1 inner join video_game v2 
on v1.platform_id = v2.platform_id 
AND v1.publisher_id = v2.publisher_id 
AND v1.year = v2.year 
AND v1.name <> v2.name 
AND v1.genre = v2.genre 
AND v1.rank < v2.rank;"""

ExecuteQuery(query)

,rank,name,rank,name
0,4,Wii Sports Resort,15,Wii Fit Plus
1,11,Nintendogs,42,Animal Crossing: Wild World


7) List all regions with their total sales in all games. 

In [23]:
query = """
select r.name, ifnull(sum(s.sales), '') 
from region r 
left join games_sales s 
on r.id = s.region_id 
group by r.name;"""

ExecuteQuery(query)

,name,"ifnull(sum(s.sales), '')"
0,European Union,284.42
1,Japan,140.54
2,North America,491.49
3,Oceania,
4,Rest of the World,94.71
5,South America,


8) Some developers (companies) publish/develop games, make platforms (consoles), or do both. List all developers with the number of games they published and the platforms they made.

In [24]:
query = """
select t1.name, t1.games, ifnull(t2.platforms,0) as platforms 
from 
  (select d.name, count(v.name) as games 
  from video_game v join developer d 
  on v.publisher_id = d.id group by d.name) t1 
  left join 
  (select d.name, count(p.platform_name) as platforms 
  from platform p join developer d 
  on p.developer_id = d.id group by d.name)t2 
  on t1.name = t2.name;	"""

ExecuteQuery(query)

,name,games,platforms
0,Activision,8,0
1,Microsoft,2,1
2,Nintendo,32,8
3,Sony,2,3
4,Take-Two Interactive,6,0


9) Find names of platforms that have racing games but have no action games.

In [25]:
query = """
select t1.platform_name from 
  (select DISTINCT(p.platform_name), v.genre as genre1 
  from video_game v join platform p 
  on v.platform_id = p.platform_id 
  where v.genre = 'Racing') t1 
LEFT JOIN 
  (select DISTINCT(p.platform_name), v.genre as genre2 
  from video_game v join platform p 
  on v.platform_id = p.platform_id 
  where v.genre = 'Action')t2 
on t1.platform_name = t2.platform_name 
where genre2 IS NULL;"""

ExecuteQuery(query)

,platform_name
0,Wii
1,3DS
